# 準備

In [ ]:
import getpass
import os

# OpenAI API キーの設定
api_key = getpass.getpass("OpenAI API キーを入力してください: ")
os.environ["OPENAI_API_KEY"] = api_key

In [ ]:
!pip install -q langchain langgraph langchain-openai langchain-community

# 4.1 マルチエージェントシステムの構築

## 4.2.2 チャットボットの構築

In [6]:
from typing_extensions import TypedDict
from typing import Annotated

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

class State(TypedDict):
    count: int
    messages: Annotated[list, add_messages]

def chatbot(state: State):
    messages = [llm.invoke(state["messages"])]
    count = state["count"] + 1
    return {
        "messages": messages,
        "count": count,
    }

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

In [ ]:
from IPython.display import display, Image

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
from langchain_core.messages import HumanMessage


human_message = HumanMessage("こんにちは")

for event in graph.stream({"messages": [human_message], "count": 0}):
    for value in event.values():
        print(f"### ターン{value['count']} ###")
        value["messages"][-1].pretty_print()

In [10]:
# ペルソナの設定

from langchain_core.messages import SystemMessage


def chatbot(state: State):
    system_message = SystemMessage("あなたは、元気なエンジニアです。元気に返答してください。")
    messages = [llm.invoke([system_message] + state["messages"])]
    count = state["count"] + 1
    return {
        "messages": messages,
        "count": count,
    }

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

In [ ]:
from langchain_core.messages import HumanMessage


human_message = HumanMessage("上手くデバッグができません")

for event in graph.stream({"messages": [human_message], "count": 0}):
    for value in event.values():
        print(f"### ターン{value['count']} ###")
        value["messages"][-1].pretty_print()

## 4.2.3 複数のエージェントの接続

### 3つのエージェントの準備

In [31]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain.prompts import SystemMessagePromptTemplate
import functools
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(model="gpt-4o")

def agent_with_persona(state: State, name: str, traits: str):
    system_message_template = SystemMessagePromptTemplate.from_template(
        "あなたの名前は{name}です。\nあなたの性格は以下のとおりです。\n\n{traits}"
    )
    system_message = system_message_template.format(name=name, traits=traits)

    message = HumanMessage(
        content=llm.invoke([system_message, *state["messages"]]).content,
        name=name,
    )

    return {
        "messages": [message],
    }

kenta_traits = """\
- アクティブで冒険好き
- 新しい経験を求める
- アウトドア活動を好む
- SNSでの共有を楽しむ
- エネルギッシュで社交的"""

mari_traits = """\
- 穏やかでリラックス志向
- 家族を大切にする
- 静かな趣味を楽しむ
- 心身の休養を重視
- 丁寧な生活を好む"""

yuta_traits = """\
- バランス重視
- 柔軟性がある
- 自己啓発に熱心
- 伝統と現代の融合を好む
- 多様な経験を求める"""

kenta = functools.partial(agent_with_persona, name="kenta", traits=kenta_traits)
mari = functools.partial(agent_with_persona, name="mari", traits=mari_traits)
yuta = functools.partial(agent_with_persona, name="yuta", traits=yuta_traits)

### 3つのエージェントが順番に回答するシステム

In [32]:
from typing_extensions import TypedDict
from typing import Annotated
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]

In [33]:
from langgraph.graph import StateGraph, START, END

graph_builder = StateGraph(State)

graph_builder.add_node("kenta", kenta)
graph_builder.add_node("mari", mari)
graph_builder.add_node("yuta", yuta)

graph_builder.add_edge(START, "kenta")
graph_builder.add_edge("kenta", "mari")
graph_builder.add_edge("mari", "yuta")
graph_builder.add_edge("yuta", END)

graph = graph_builder.compile()

In [ ]:
from IPython.display import display, Image

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
from langchain_core.messages import HumanMessage

human_message = HumanMessage("休日の過ごし方について、建設的に議論してください。")

for event in graph.stream({"messages": [human_message]}):
    for value in event.values():
        value["messages"][-1].pretty_print()

### 3つのエージェントが一斉に回答するシステム

In [37]:
from langgraph.graph import StateGraph, START, END

graph_builder = StateGraph(State)

graph_builder.add_node("kenta", kenta)
graph_builder.add_node("mari", mari)
graph_builder.add_node("yuta", yuta)


graph_builder.add_edge(START, "kenta")
graph_builder.add_edge(START, "mari")
graph_builder.add_edge(START, "yuta")
graph_builder.add_edge("kenta", END)
graph_builder.add_edge("mari", END)
graph_builder.add_edge("yuta", END)

graph = graph_builder.compile()

In [ ]:
from IPython.display import display, Image

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
from langchain_core.messages import HumanMessage


human_message = HumanMessage("休日の過ごし方について、建設的に議論してください。")

for event in graph.stream({"messages": [human_message]}):
    for value in event.values():
        value["messages"][-1].pretty_print()

### 3つのエージェントから選択されたエージェントが回答するシステム

In [41]:
from pydantic import BaseModel, Field
from langchain.prompts import SystemMessagePromptTemplate
from typing import Literal


class State(TypedDict):
    messages: Annotated[list, add_messages]
    next: str

member_dict = {
    "kenta": kenta_traits,
    "mari": mari_traits,
    "yuta": yuta_traits,
}

#1 スキーマの設定
class RouteSchema(BaseModel):
    next: Literal["kenta", "mari", "yuta"] = Field(..., description="次に発言する人")

#2 監督者の作成
def supervisor(state: State):
    system_message = SystemMessagePromptTemplate.from_template(
        "あなたは以下の作業者間の会話を管理する監督者です：{members}。"        "各メンバーの性格は以下の通りです。"        "{traits_description}"        "与えられたユーザーリクエストに対して、次に発言する人を選択してください。"    )

    members = ", ".join(list(member_dict.keys()))
    traits_description = "\n".join([f"**{name}**\n{traits}" for name, traits in member_dict.items()])

    system_message = system_message.format(members=members, traits_description=traits_description)

    llm_with_format = llm.with_structured_output(RouteSchema)

    next = llm_with_format.invoke([system_message] + state["messages"]).next
    return {"next": next}


In [42]:
graph_builder = StateGraph(State)

graph_builder.add_node("supervisor", supervisor)
graph_builder.add_node("kenta", kenta)
graph_builder.add_node("mari", mari)
graph_builder.add_node("yuta", yuta)

graph_builder.add_edge(START, "supervisor")
graph_builder.add_conditional_edges(
    "supervisor",
    lambda state: state["next"],
    {"kenta": "kenta", "mari": "mari", "yuta": "yuta"},
)

for member in ["kenta", "mari", "yuta"]:
    graph_builder.add_edge(member, END)

graph = graph_builder.compile()

In [ ]:
from IPython.display import display, Image

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
from langchain_core.messages import HumanMessage

human_message = HumanMessage("休日のまったりした過ごし方を教えて")
for event in graph.stream({"messages": [human_message]}):
    for value in event.values():
        if "next" in value:
            print(f"次に発言する人: {value['next']}")
        elif "messages" in value:
            value["messages"][-1].pretty_print()

## 4.2.4 ツールの使用

In [ ]:
import getpass
import os

# Tavily API キーの設定
api_key = getpass.getpass("Tavily API キーを入力してください: ")
os.environ["TAVILY_API_KEY"] = api_key

In [25]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict
from typing import Annotated
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

#1 ツールの作成
tavily_tool = TavilySearchResults(max_results=2)

#2 ツールの紐づけ
llm = ChatOpenAI(model="gpt-4o")
llm_with_tool = llm.bind_tools([tavily_tool])

#3 ツールを使ったチャットボットの作成
def chatbot(state: State):
    messages = [llm_with_tool.invoke(state["messages"])]
    return {
        "messages": messages,
    }

In [26]:
import json

from langchain_core.messages import ToolMessage


class ToolNode:
    def __init__(self, tools: list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tools}

    def __call__(self, state: State):
        #1 最後のメッセージを取得
        if messages := state.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("入力にメッセージが見つかりません")

        #2 ツールの実行
        tool_messages = []
        for tool_call in message.tool_calls:
            #2.1 エージェントが指定したnameとargsを元にツールを実1行
            tool_result = self.tools_by_name[tool_call["name"]].invoke(
                tool_call["args"]
            )
            #2.2 ツールの実行結果をメッセージとして追加
            tool_messages.append(
                ToolMessage(
                    content=json.dumps(tool_result, ensure_ascii=False),
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"],
                )
            )

        return {
            "messages": tool_messages,
        }

tool_node = ToolNode([tavily_tool])

In [27]:
from typing import Literal

def route_tools(
    state: State,
) -> Literal["tools", "__end__"]:
    if messages := state.get("messages", []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"stateにツールに関するメッセージが見つかりませんでした: {state}")

    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    return "__end__"

In [28]:
from langgraph.graph import StateGraph, START, END

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    route_tools,
    ["tools", "__end__"],
)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile()

In [ ]:
from IPython.display import display, Image

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
from langchain_core.messages import HumanMessage

human_message = {
    "messages": [HumanMessage("今日の東京の天気を教えて")],
    "count": 0,
}

for event in graph.stream(human_message):
    for value in event.values():
        value["messages"][-1].pretty_print()